In [14]:
import os
import xarray as xr
import pygrib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

In [15]:
# Function to read GRIB file and extract data
def read_grib_file(grib_file):
    grbs = pygrib.open(grib_file)
    grb = grbs.select()[0]

    # Extract the data from the GRIB message
    data = grb.values
    latitudes, longitudes = grb.latlons()

    grbs.close()

    return data, latitudes, longitudes

In [16]:
def preprocess_data(data):
    # Normalize data (optional, based on your requirements)
    normalized_data = (data - np.mean(data)) / np.std(data)

    return normalized_data


In [17]:
# Function to build and train the CNN model
def build_cnn_model(input_shape):
    model = Sequential()

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Output layer for regression

    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

In [18]:
def load_data(folder_path):
    subfolders = ['000', '012']
    X = []
    Y = []

    for subfolder in subfolders:
        subfolder_path = os.path.join(folder_path, subfolder)
        files = os.listdir(subfolder_path)

        for file in files:
            grib_file_12h = os.path.join(subfolder_path, file)
            grib_file_0h = os.path.join(folder_path, '000', file)

            if os.path.isfile(grib_file_12h) and os.path.isfile(grib_file_0h):
                # Read and preprocess the data
                data_12h, latitudes, longitudes = read_grib_file(grib_file_12h)
                data_0h, _, _ = read_grib_file(grib_file_0h)

                # Preprocess the data
                X.append(preprocess_data(data_12h))
                Y.append(preprocess_data(data_0h))

    return np.array(X), np.array(Y)


In [19]:
data_folder = '../download'  # Folder path containing the GRIB files


In [20]:
# Load the data
X, Y = load_data(data_folder)


In [21]:
print(len(Y))
print(len(X))

640
640


In [32]:
print(X[2])

[[-1.45527329 -1.45527329 -1.45527329 ... -1.45527329 -1.45527329
  -1.45527329]
 [-1.45605818 -1.45587953 -1.45569502 ... -1.45660291 -1.45642133
  -1.45623975]
 [-1.45241781 -1.45203122 -1.45164756 ... -1.45358343 -1.45319098
  -1.4528044 ]
 ...
 [-0.8939508  -0.89369015 -0.89342364 ... -0.89470641 -0.8944604
  -0.89420853]
 [-0.85844038 -0.85831737 -0.85819144 ... -0.85880061 -0.85868346
  -0.85856338]
 [-0.82595529 -0.82595529 -0.82595529 ... -0.82595529 -0.82595529
  -0.82595529]]


In [22]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



In [23]:
# Print the shape of X_train for debugging
print(X_train.shape)  
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(512, 361, 720)
(512, 361, 720)
(128, 361, 720)
(128, 361, 720)


In [24]:
# Build the CNN model
input_shape = X_train.shape[1:]

model = build_cnn_model(input_shape)



In [27]:
print(input_shape)

(361, 720)


In [11]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=4)



Epoch 1/10


ValueError: in user code:

    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, samp

In [ ]:
# Make predictions (replace X_test with your test data)
#predictions = model.predict(X_test)
#predictions



In [ ]:
# Perform any further processing or analysis with the predictions
# Hereafter, you can do whatever you want.

/home/data/detectives/data-detectives
